# Microsoft Entra ID Administration Agent (MEAN)
---
* Collaborators:
    * Roberto Rodriguez (@Cyb3rWard0g)
    * Nestori Syynimaa  (@DrAzureAD)
* References:
    * https://python.langchain.com/docs/use_cases/apis
    * https://python.langchain.com/docs/integrations/toolkits/openapi
    * https://learn.microsoft.com/en-us/graph/api/resources/users?view=graph-rest-1.0

## Download MS Graph (User) OpenAPI specification

Location: https://github.com/microsoftgraph/msgraph-sdk-powershell/blob/dev/openApiDocs/v1.0/Users.yml

## Load Open API Specification

In [1]:
import os
import yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

current_directory = os.path.abspath('')
data_directory = os.path.join(current_directory, "openapi-specs")
msgraph_api_file = os.path.join(data_directory, "OpenAPI-MSGraph-v1.0-Users_dereferenced.yml")

raw_msgraph_api_spec = yaml.load(open(msgraph_api_file,encoding='utf-8').read(), Loader=yaml.Loader)
msgraph_api_spec = reduce_openapi_spec(raw_msgraph_api_spec)

## Define LangChain Authorization Header Function

In [2]:
from azure.identity import InteractiveBrowserCredential
from langchain.requests import RequestsWrapper

def construct_auth_headers(raw_spec: dict):
    credential = InteractiveBrowserCredential()
    access_token = (credential.get_token("https://graph.microsoft.com/.default")).token
    return {"Authorization": f"Bearer {access_token}",
           "ConsistencyLevel": "eventual"}


# Get API credentials.
headers = construct_auth_headers(raw_msgraph_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

## MS Graph API Spec

In [3]:
endpoints = [
    (route, operation)
    for route, operations in raw_msgraph_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post", "patch"]
]
len(endpoints)

5

## Initialize Open AI Key

In [4]:
import os
from dotenv import load_dotenv
import openai

# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Define OpenAI Chat API

In [5]:
from langchain_openai import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner

llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
msgraph_agent = planner.create_openapi_agent(msgraph_api_spec, requests_wrapper, llm, handle_parsing_errors=True)

## Run Query

In [6]:
user_query = (
    "Show lastPasswordChangeDateTime and jobTitle of users."
)
msgraph_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: Show lastPasswordChangeDateTime and jobTitle of users.
Observation: 1. GET /users to retrieve a list of user objects.
2. For each user object, extract the 'lastPasswordChangeDateTime' and 'jobTitle' properties.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /users to retrieve a list of user objects.
2. For each user object, extract the 'lastPasswordChangeDateTime' and 'jobTitle' properties.

> Entering new AgentExecutor chain...
I need to make a GET request to the /users endpoint. I will set the $top parameter to 50 to limit the number of results. I will also set the $select parameter to 'lastPasswordChangeDateTime' and 'jobTitle' to only retrieve these properties for each user. I don't need to use the $search, $filter, $orderby, or $expand parameters for this request.

Action: requests_get
Action Input: 
{
  "url": "https://graph.microsoft.com/v1.0/users",
  "params": {
 

{'input': 'Show lastPasswordChangeDateTime and jobTitle of users.',
 'output': "Here are the 'lastPasswordChangeDateTime' and 'jobTitle' for each user:\n\n1. User ID: 8ca16a47-fec3-465c-9b4c-64430250bba9, Last Password Change: 2021-07-16T12:26:22Z, Job Title: null\n2. User ID: 449162e2-19cf-4417-aa66-371e8a3cdbdd, Last Password Change: 2023-06-13T15:11:42Z, Job Title: null\n3. User ID: 513bc81a-f972-434b-a3f5-6895ea1f8d81, Last Password Change: 2023-08-28T09:22:59Z, Job Title: null\n4. User ID: d9e58c5e-2c1f-48d4-ac0c-c04afabd4c5b, Last Password Change: 2024-01-30T15:14:27Z, Job Title: null\n5. User ID: 7a5ac4a1-f6d5-4f0a-a3c4-a844c44ac48a, Last Password Change: 2024-02-21T17:14:41Z, Job Title: null\n6. User ID: 1eac2056-2e03-4cb8-9110-22d1d87d2a14, Last Password Change: 2023-03-13T13:59:13Z, Job Title: null\n7. User ID: b5f58c9d-06ec-41b1-afb7-04c90453c5bb, Last Password Change: 2024-02-28T12:37:37Z, Job Title: null\n8. User ID: 2ff693d0-96bd-4a1c-9bc1-c2cc25c560f5, Last Password Chan

## One Cell Execution

In [7]:
# Download MS Graph API OpenAPI Specification:
#  https://github.com/microsoftgraph/msgraph-sdk-powershell/blob/dev/openApiDocs/v1.0/Users.yml
import os
import yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

# Read File Locally
current_directory = os.path.abspath('')
msgraph_api_file = os.path.join('openapi-specs', "OpenAPI-MSGraph-v1.0-Users_dereferenced.yml")
raw_msgraph_api_spec = yaml.load(open(msgraph_api_file,encoding='utf-8').read(), Loader=yaml.Loader)

# Read as OpenAPI Specification
msgraph_api_spec = reduce_openapi_spec(raw_msgraph_api_spec)

# Define LangChain Authorization Header Function 
from azure.identity import InteractiveBrowserCredential
from langchain.requests import RequestsWrapper

def construct_auth_headers(raw_spec: dict):
    credential = InteractiveBrowserCredential()
    access_token = (credential.get_token("https://graph.microsoft.com/.default")).token
    return {"Authorization": f"Bearer {access_token}",
           "ConsistencyLevel": "eventual"}

# Get Access Token -> Authorization Header.
headers = construct_auth_headers(raw_msgraph_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

# Define OpenAI Chat API
from langchain_openai import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner
from dotenv import load_dotenv
import openai
# Initialize OpenAI Key. Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

# Create OpenAPI Agent
msgraph_agent = planner.create_openapi_agent(msgraph_api_spec, requests_wrapper, llm, handle_parsing_errors=True)

# Pass prompt (Natural Language) to Agent -> Call MS Graph API
msgraph_agent.invoke("Show lastPasswordChangeDateTime and jobTitle of users.")



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: Show lastPasswordChangeDateTime and jobTitle of users.
Observation: 1. GET /users to retrieve a list of user objects.
2. For each user object, extract the 'lastPasswordChangeDateTime' and 'jobTitle' properties.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /users to retrieve a list of user objects.
2. For each user object, extract the 'lastPasswordChangeDateTime' and 'jobTitle' properties.

> Entering new AgentExecutor chain...
I need to make a GET request to the /users endpoint. I will set the $top parameter to 50 to limit the number of results. I will also set the $select parameter to 'lastPasswordChangeDateTime' and 'jobTitle' to only retrieve these properties for each user. I don't need to use the $search, $filter, $orderby, or $expand parameters for this task.

Action: requests_get
Action Input: 
{
  "url": "https://graph.microsoft.com/v1.0/users",
  "params": {
    

{'input': 'Show lastPasswordChangeDateTime and jobTitle of users.',
 'output': 'Here is the extracted information for \'lastPasswordChangeDateTime\' and \'jobTitle\':\n\n1. {"lastPasswordChangeDateTime":"2021-07-16T12:26:22Z","jobTitle":null}\n2. {"lastPasswordChangeDateTime":"2023-06-13T15:11:42Z","jobTitle":null}\n3. {"lastPasswordChangeDateTime":"2023-08-28T09:22:59Z","jobTitle":null}\n4. {"lastPasswordChangeDateTime":"2024-01-30T15:14:27Z","jobTitle":null}\n5. {"lastPasswordChangeDateTime":"2024-02-21T17:14:41Z","jobTitle":null}\n6. {"lastPasswordChangeDateTime":"2023-03-13T13:59:13Z","jobTitle":null}\n7. {"lastPasswordChangeDateTime":"2024-02-28T12:37:37Z","jobTitle":null}\n8. {"lastPasswordChangeDateTime":"2021-07-01T14:42:46Z","jobTitle":null}\n9. {"lastPasswordChangeDateTime":"2023-08-22T08:34:38Z","jobTitle":"Marketing Assistant"}\n10. {"lastPasswordChangeDateTime":"2023-08-23T10:13:29Z","jobTitle":"IT Admin"}\n\nPlease note that the \'id\' field is missing in the API response